1. Clustering

Usa el conjunto de datos de las viviendas en California que puedes descargar con la función: sklearn.datasets.fetch_california_housing

En el conjunto de datos cada fila es un “bloque” que es la mínima unidad geográfica para la cual el censo estadounidense hace públicos los datos (suele tener desde varios cientos a unos pocos miles de habitantes). 

El objetivo es realizar agrupamientos de los bloques que presenten características similares. No incluyas la latitud y longitud para no agrupar los bloques por su cercanía geográfica, sino por sus características. Tampoco incluyas la variable target del dataset que es la mediana del precio de las viviendas en el bloque en cuestión. 

In [44]:
import numpy as np
import sklearn
import statistics as stat
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.datasets.samples_generator import make_blobs
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

np.random.seed(11)  

# conjunto de datos de las viviendas en California
# .data, .target, .feature_names, .DESCR
cali = sklearn.datasets.fetch_california_housing()

# descripcion del conjunto
print(cali.DESCR)


.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

Por cada conjunto de datos que utilices deberás incluir una breve descripción del mismo
    - Nombre del conjunto de datos
    - Breve descripción del problema que describe
    - Nombre y tipo de las variables
    - Estadísticos descriptivos de cada variable

1) Describe el conjunto de datos tal y como se indica más arriba y extrae algunas conclusiones de las variables, su distribución y su correlación
   
    - Nombre del conjunto de datos
        California Housing Dataset
        
    - Breve descripción del problema que describe
        Representar bloques de zonas habitables en California de 1990. Un bloque tiene entre 600 y 3.000 personas.

    - Nombre y tipo de las variables
        Proporciona los siguientes datos :
            MedInc    
            HouseAge  
            AveRooms  
            AveBedrms 
            Population
            AveOccup  
            Latitude  
            Longitude 
         
        todas las variables son de tipo numpy.float64
        
    - Estadísticos descriptivos de cada variable
        MedInc     		mediana de los ingresos del bloque
        HouseAge  		mediana de las edades de las casas del bloque
        AveRooms  		número medio de habitaciones
        AveBedrms 		número medio de dormitorios
        Population 		población del bloque
        AveOccup   		media de residentes por vivienda
        Latitude   		latitud del bloque
        Longitude		longitud del bloque
        
        MedInc : promedio: 37.19233650793651
        HouseAge : promedio: 294.40765742530755
        AveRooms : promedio: 60.3789052259887
        AveBedrms : promedio: 67.58518202054795
        Population : promedio: 68.2488251930502
        AveOccup : promedio: 49.08024766839378
        Latitude : promedio: 134.15759664396887
        Longitude : promedio: 141.91970054095827
        
        MedInc : mediana: 7.654663492063492
        HouseAge : mediana: 7.269768541300527
        AveRooms : mediana: 7.77276779661017
        AveBedrms : mediana: 5.7302257990867576
        Population : mediana: 5.064026640926641
        AveOccup : mediana: 4.399229015544042
        Latitude : mediana: 4.2955033073929965
        Longitude : mediana: 3.9587635239567236
        
        MedInc : desviación típica: 125.81140884875056
        HouseAge : desviación típica: 852.5859304471638
        AveRooms : desviación típica: 183.6381187178246
        AveBedrms : desviación típica: 204.93098490132294
        Population : desviación típica: 207.40011377204837
        AveOccup : desviación típica: 156.0331858478623
        Latitude : desviación típica: 391.32996492058425
        Longitude : desviación típica: 413.458823804565
        
        MedInc : varianza: 15828.51059650747
        HouseAge : varianza: 726902.768796456
        AveRooms : varianza: 33722.95864622184
        AveBedrms : varianza: 41996.70857262625
        Population : varianza: 43014.807192658605
        AveOccup : varianza: 24346.355085833544
        Latitude : varianza: 153139.14144474568
        Longitude : varianza: 170948.1989818543
        


68.2488251930502
207.40011377204837
43014.807192658605


2) Considera si debes re‐escalar las variables antes y el tipo de escalado que usas. Razona tu elección. 

In [133]:
X = []
pop = 0
kel = 0
for i in range(0, len(cali.data)):
    pop = pop + cali.data[i][4]
    kel = kel + cali.data[i][5]
    X.append(cali.data[i][5])
pop = pop / len(cali.data)
kel = kel / len(cali.data)
print(pop)
print(kel)

print(stat.mean(X))
  
comparar plots para decidir 2)

1425.4767441860465
3.070655159436382
3.0706551594363742


In [58]:
def plot_clusters(X, labels=None, centers=None):
    """ Función auxiliar para pintar los puntos de los clusters y, optativamente, sus centros.
    :param X: array de puntos de dos dimensiones (array de array de enteros)
    :param labels: cluster al que pertenece cada punto (array de enteros)
    :param centers: coordenadas de los centroides de los clusters (array de array de enteros)
    """
    
    colors = ['r','g','b','c','y','k','w','m',]
    fig = plt.figure(figsize=(8,8))
    
    # Los dos primeros parámetros de la función scatter son las coordenadas de los puntos,
    # el parámetro 'c' indica la clase asignada de cada punto, y 'cmap' se usa para colorear 
    # las distintas clases
    plt.scatter(X[:,0], X[:,1], c=labels, cmap=ListedColormap(colors))
    
    # Pintar los centroides de los clusters
    if centers is not None:
        plt.scatter(centers[:,0], centers[:,1], marker="x", color='k', s=150, linewidths = 5, zorder=10)
    plt.show()

    
# Crear una instancia del agrupador k-means
km = KMeans(init='random', n_clusters=3)

# Agrupar los puntos en clusters
km.fit(X)

# Podemos consultar el cluster asignado a cada punto y el centro de cada cluster
labels = km.labels_
centers = km.cluster_centers_

plot_clusters(X, labels, centers)

In [59]:
plot_clusters(cali.data[:][5], labels)

NameError: name 'labels' is not defined

3) Aplica el algoritmo de clustering k‐medias y determina el número de clusters que consideras adecuado para el conjunto de datos, justificando tu elección.

4) Trata de averiguar qué representa cada uno de los clusters que has obtenido. Si en el apartado anterior has obtenido más de 5 clusters, basta con que comentes los dos más numerosos y los dos menos numerosos. ¿Qué valores toman las variables en cada cluster? Puedes usar estadísticos descriptivos Te recomendamos que uses las variables en su escala original y no en la transformada (ya que se interpretará mucho peor).

5) Pinta los clusters en un gráfico de dispersión en función de dos de las variables de entrada que consideres interesantes. ¿Ves que considerando solamente esas dos variables se diferencien bien algunos de los clusters? ¿Cuáles? Por el contrario, ¿cuáles se confuden más?  Te recomendamos que en este caso uses las variables tal y como las usaste para hacer el clustering (es decir, re‐escaladas, si las re‐escalastes) para así diferenciar los clusters mejor 